In [1]:
import pandas as pd
import glob
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib.dates import DateFormatter
import matplotlib as mpl
import re
import pickle
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
import sklearn.metrics as metrics
from pandas.api.types import CategoricalDtype
import xgboost as xgb
from skopt import BayesSearchCV


In [2]:
BASE_PATH = "../data"
RAW_DATA_PATH = f"{BASE_PATH}/raw"
TMP_PATH = f"{BASE_PATH}/tmp"
FILE_PATTERN = RAW_DATA_PATH + "/*/OD*"
STATION_PATTERN = RAW_DATA_PATH + "/*/Stations*"
START_STATION = 6184
END_STATION = 6015
START_DATE = "2014-04-15"
END_DATE = "2017-11-15"
THOUSANDS_SYMBOL = " (000s)"
USE_CACHE = True
NUM_CLUSTER = 12
MINIMUM_TRIPS_PAIRWISE = 800
FORECAST_START = "2017-09-04"
FORECAST_END = "2017-09-10"
ITERATIONS = 1 # 1000


In [3]:
def load_data(path_pattern: str, parse_dates=False) -> pd.DataFrame:
    filepath = [pd.read_csv(name, parse_dates=parse_dates) for name in glob.glob(path_pattern)]
    df = pd.concat(filepath)
    return df

In [4]:
df = load_data(FILE_PATTERN, parse_dates=["start_date", "end_date"])
df = df.loc[
        (df.start_date >= START_DATE) &
        (df.end_date < END_DATE)
    ].reset_index(drop=True)

In [5]:
stations = pd.read_csv(f"{TMP_PATH}/clean_stations.csv")
cluster = pd.read_csv(f"{TMP_PATH}/station_cluster.csv")

In [6]:
stations = stations.merge(cluster, on="code")
stations.rename({"code": "start_station_code"}, axis=1, inplace=True)
stations.head()

,start_station_code,name,latitude,longitude,cluster
0,6347,Métro St-Michel (Shaughnessy / St-Michel),45.559199,-73.599658,4
1,6219,de l'Hôtel-de-Ville / Roy,45.517333,-73.574436,5
2,6312,de Kent / de la Côte-des-Neiges,45.501302,-73.633161,6
3,6138,Gauthier / De Lorimier,45.531818,-73.565317,8
4,6205,Milton / du Parc,45.509710,-73.573540,1


In [7]:
combined_df = df.merge(stations[["start_station_code", "latitude", "longitude", "cluster"]], on="start_station_code", how="inner")

In [8]:
combined_df.head()

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member,latitude,longitude,cluster
0,2014-08-01 00:00:00,6215,2014-08-01 00:11:00,6151,702,0,45.514914,-73.578243,5
1,2014-08-01 00:03:00,6215,2014-08-01 00:16:00,6152,800,1,45.514914,-73.578243,5
2,2014-08-01 00:09:00,6215,2014-08-01 00:12:00,6181,178,1,45.514914,-73.578243,5
3,2014-08-01 06:15:00,6215,2014-08-01 06:19:00,6221,233,1,45.514914,-73.578243,5
4,2014-08-01 06:49:00,6215,2014-08-01 06:57:00,6065,512,1,45.514914,-73.578243,5


In [9]:
combined_df.shape

(14806753, 9)

In [10]:
def regression_results(y_true, y_pred):
    # Regression metrics
    explained_variance=metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_true, y_pred) 
    mse=metrics.mean_squared_error(y_true, y_pred) 
    mean_squared_log_error=metrics.mean_squared_log_error(y_true, y_pred)
    median_absolute_error=metrics.median_absolute_error(y_true, y_pred)
    r2=metrics.r2_score(y_true, y_pred)
    print('explained_variance: ', round(explained_variance,4))    
    print('mean_squared_log_error: ', round(mean_squared_log_error,4))
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))

# Prediction member only

In [11]:
clean_df_member = combined_df.loc[combined_df.is_member == 1].copy()

In [12]:
clean_df_member.shape

(12302081, 9)

**Data Cleaning**: since our training unit should be number of trips between pairwise-stations per day, need to do some basic transform


In [13]:
clean_df_member["start_date"] = clean_df_member["start_date"].dt.normalize()
clean_df_member.drop(["end_date"], axis=1, inplace=True)
clean_df_member = clean_df_member.groupby(["start_date", "start_station_code", "end_station_code"]).agg(
    {
        "duration_sec": np.mean, 
        "is_member": np.mean, 
        "latitude": max, 
        "longitude": max, 
        "cluster": [max, "count"]
    }
).reset_index()
clean_df_member.columns = ["start_date", "start_station_code", "end_station_code", "avg_duration", "is_member_ratio", "latitude", "longitude", "cluster", "num_trips"]

In [14]:
train = clean_df_member.loc[clean_df_member.start_date <= '2017-07-30']
test = clean_df_member.loc[clean_df_member.start_date > '2017-07-30']
forecast = clean_df_member.loc[
    (clean_df_member.start_date >= FORECAST_START) & 
    (clean_df_member.start_date <= FORECAST_END) &
    (clean_df_member.start_station_code == START_STATION) &
    (clean_df_member.end_station_code == END_STATION)
]

In [15]:
train.shape, test.shape, forecast.shape # tradition split train test, with the forecast set for the requirement

((7874799, 9), (1404420, 9), (5, 9))

In [16]:
prediction_range = pd.date_range(start=FORECAST_START, end=FORECAST_END)
missing_prediction = list(set(prediction_range) - set(forecast.start_date))

# add back missing prediction
forecast = pd.concat([forecast, pd.DataFrame({
    "start_date":missing_prediction
})], axis=0).reset_index(drop=True)
forecast[["start_station_code", "end_station_code", "latitude", "longitude", "cluster"]] = forecast[["start_station_code", "end_station_code", "latitude", "longitude", "cluster"]].fillna(method="ffill")
forecast[["num_trips", "avg_duration", "is_member_ratio"]] = forecast[["num_trips", "avg_duration", "is_member_ratio"]].fillna(0)
forecast.sort_values("start_date", inplace=True)
forecast

,start_date,start_station_code,end_station_code,avg_duration,is_member_ratio,latitude,longitude,cluster,num_trips
6,2017-09-04,6184.0,6015.0,0.000000,0.0,45.524673,-73.58255,2.0,0.0
0,2017-09-05,6184.0,6015.0,576.166667,1.0,45.524673,-73.58255,2.0,6.0
5,2017-09-06,6184.0,6015.0,0.000000,0.0,45.524673,-73.58255,2.0,0.0
1,2017-09-07,6184.0,6015.0,810.500000,1.0,45.524673,-73.58255,2.0,2.0
2,2017-09-08,6184.0,6015.0,557.333333,1.0,45.524673,-73.58255,2.0,3.0
3,2017-09-09,6184.0,6015.0,525.333333,1.0,45.524673,-73.58255,2.0,3.0
4,2017-09-10,6184.0,6015.0,1374.000000,1.0,45.524673,-73.58255,2.0,1.0


**Feature Engineering**
<br>
Create feature ride frequency per route, dropping low category because the model will prone to predict 0 for those entries, and make the training faster

In [17]:
trip_labels = ["low", "medium", "high"] # low: average daily ride less than 1, medium: between 1-2, high: > 2
trip_numbers = train.groupby(["start_station_code", "end_station_code"])["num_trips"].sum().reset_index()
trip_numbers["route_freq"] = "low"
trip_numbers.loc[trip_numbers.num_trips > 750, "route_freq"] = "medium"
trip_numbers.loc[trip_numbers.num_trips > 1500, "route_freq"] = "high"
trip_numbers.drop("num_trips", inplace=True, axis=1)
trip_numbers = trip_numbers.loc[trip_numbers.route_freq != "low"] 


Create feature for the average length of the ride, the purpose is to capture travel behavior, longer might mean leisure ride.

In [41]:
route_length = ["1", "2", "3", "4"] #higher the longer
trip_length = pd.qcut(train.groupby(["start_station_code", "end_station_code"])["avg_duration"].mean(), q=4, labels=route_length).reset_index()
trip_length.columns = ["start_station_code", "end_station_code", "route_length"]

In [42]:
train = train.merge(trip_numbers, on=["start_station_code", "end_station_code"])
train = train.merge(trip_length, on=["start_station_code", "end_station_code"])

test = test.merge(trip_numbers, on=["start_station_code", "end_station_code"])
test = test.merge(trip_length, on=["start_station_code", "end_station_code"])

forecast = forecast.merge(trip_numbers, on=["start_station_code", "end_station_code"])
forecast = forecast.merge(trip_length, on=["start_station_code", "end_station_code"])

**Feature Selection**

In [43]:
def feature_selection(dataframe, is_train=False, categories=None):
    
    X_train = dataframe[["start_date", "cluster", "route_freq", "route_length"]].copy()
    y_train = dataframe["num_trips"]

    X_train["year"] = X_train.start_date.dt.year
    X_train["month"] = X_train.start_date.dt.month
    X_train["day_of_week"] = X_train.start_date.dt.day_of_week
    X_train.drop("start_date", axis=1, inplace=True)
    
    if is_train:
        X_train = X_train.astype("category")
        categories = {
                    col: list(X_train[col].cat.categories)
                    for col in X_train.columns
                }
        X_train_ohe = pd.get_dummies(X_train, drop_first=True)
        return X_train_ohe, y_train, categories
    
    for k, v in categories.items():
        X_train[k] = X_train[k].astype(CategoricalDtype(categories=v))    
        X_train_ohe = pd.get_dummies(X_train, drop_first=True)
    return X_train_ohe, y_train

In [44]:
X_train_ohe, y_train, categories = feature_selection(train, is_train=True)
X_test_ohe, y_test = feature_selection(test, is_train=False, categories=categories)
X_forecast_ohe, y_forecast = feature_selection(forecast, is_train=False, categories=categories)

In [45]:
model = xgb.XGBRegressor(objective ='reg:squarederror', n_estimators = 100)
model.fit(X_train_ohe,y_train)
y_true = y_test.values
y_pred = model.predict(X_test_ohe.values)
regression_results(y_true, y_pred)

explained_variance:  0.2126
mean_squared_log_error:  0.1798
r2:  0.2052
MAE:  1.3244
MSE:  3.6534
RMSE:  1.9114


In [46]:
y_true = y_forecast.values
y_pred = model.predict(X_forecast_ohe.values)
regression_results(y_true, y_pred)

explained_variance:  -0.0344
mean_squared_log_error:  0.8616
r2:  -0.6602
MAE:  2.1365
MSE:  6.3697
RMSE:  2.5238


Daily forcast for the required period

In [47]:
y_pred, y_true

(array([3.7380257, 4.096277 , 4.2426906, 4.1975923, 3.8962386, 2.8493266,
        2.826318 ], dtype=float32),
 array([0., 6., 0., 2., 3., 3., 1.]))

# Training and CV Pipeline (DID NOT RUN)
we can add as many as models to the pipeline as we want, need three components, (model name, model object, hyperparamers search space) <br>
for illustration purpose, I only set the iteration to 1, for better prediction performance, please select more iterations. 

In [52]:
results = []
models = []
models.append(('LR', Ridge(), {'alpha': (0.01, 1.0, 'uniform')}))
models.append(('RF', RandomForestRegressor(), {
                            'n_estimators': (50, 100),        
                            'min_samples_leaf': (0, 10),
                            'min_samples_split': (0, 10),
                            'max_depth': (0, 50)})
                ) # Ensemble method - collection of many decision trees
models.append(('XGB', xgb.XGBRegressor(
                            n_jobs = 1,
                            objective = 'reg:squarederror',
                            eval_metric = 'rmse',
                            tree_method='approx'
                            ),{
                            'learning_rate': (0.01, 1.0, 'log-uniform'),
                            'min_child_weight': (0, 10),
                            'max_depth': (0, 50),
                            'max_delta_step': (0, 20),
                            'subsample': (0.01, 1.0, 'uniform'),
                            'colsample_bytree': (0.01, 1.0, 'uniform'),
                            'colsample_bylevel': (0.01, 1.0, 'uniform'),
                            'reg_lambda': (1e-9, 1000, 'log-uniform'),
                            'reg_alpha': (1e-9, 1.0, 'log-uniform'),
                            'gamma': (1e-9, 0.5, 'log-uniform'),
                            'min_child_weight': (0, 5),
                            'n_estimators': (50, 100),
                            'scale_pos_weight': (1e-6, 500, 'log-uniform')
                        }
              ))

In [ ]:
def status_print(optim_result):
    """Status callback durring bayesian hyperparameter search"""
    
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    
    
    # Get current parameters and the best parameters    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    print('Model #{}\nBest rmse: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 4),
        bayes_cv_tuner.best_params_
    ))
    
    # Save all model results
    clf_name = bayes_cv_tuner.estimator.__class__.__name__
    all_models.to_csv(f"{TMP_PATH}/{clf_name}_cv_results.csv")


for name, model, search in models: 
    bayes_cv_tuner = BayesSearchCV(
        estimator = model,
        search_spaces = search,    
        scoring = 'neg_root_mean_squared_error',
        cv = TimeSeriesSplit(n_splits=3),
        n_jobs = 4,
        n_iter = ITERATIONS,   
        verbose = 0,
        refit = True,
        random_state = 42
    )
    result = bayes_cv_tuner.fit(X_train_ohe.values, y_train.values, callback=status_print)
    results.append(result)

In [ ]:
for model in results: 
    y_true = y_test.values
    y_pred = model.predict(X_test_ohe.values)
    regression_results(y_true, y_pred)

In [ ]:
for model in results: 
    y_true = y_forecast.values
    y_pred = model.predict(X_forecast_ohe.values)
    regression_results(y_true, y_pred)